# AR classification demo

In [1]:
mapping = {
    0: 'sitting',
    1: 'standing',
    2: 'walking',
    3: 'running',
    4: 'biking'
}

In [ ]:
def decode(index):
    return mapping[index]

#### Setup cluster

In [3]:
import grpc
from hydrosdk import Cluster, Application

In [4]:
cluster = Cluster(
    http_address="<hydrosphere-http-address>",
    grpc_address="<hydrosphere-grpc-address>",
    ssl=True,                                         # turn off, if your Hydrosphere instance doesn't have
    grpc_credentials=grpc.ssl_channel_credentials()   # TLS certificates installed
)

In [5]:
app = Application.find(cluster, "<application-name>")
app.lock_while_starting()
predictor = app.predictor()

# Normal case

Use test sample from the training domain (the model was trained on data collected from sensor on hip position).

In [6]:
import os
import numpy as np
from pathlib import Path

#### Read data sample

In [8]:
path = Path(os.getcwd()).parent
X_test = np.load(f'{path}/data/SHL5hip_300_X_test.npy')[:,0]
Y_test = np.load(f'{path}/data/SHL5hip_300_Y_test.npy')[:,0]

#### Send request to the application

In [10]:
sample = np.array(X_test)[1].reshape(1,300,3,3).astype("double")
result = predictor.predict({"x": sample})

In [14]:
print(f'Predicted activity: {decode(result["y"].argmax())}')
print(f'Actual activity: {decode(np.array(Y_test)[1].argmax())}')

Predicted activity: walking
Actual activity: walking


# Outlier case

Use sample from another data domain. Data collected from another sensor position (hand) in our case.

#### Read data sample

In [16]:
X_test2 = np.load(f'{path}/data/SHL5hand_300_X_test.npy')[:,0]
Y_test2 = np.load(f'{path}/data/SHL5hand_300_Y_test.npy')[:,0]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tidylobster/Projects/proj-hydrosphere/hydro-serving-example/examples/activity_recognition/data/SHL5hand_300_X_test.npy'

#### Send request to the application

In [15]:
sample = np.array(X_test2)[1].reshape(1,300,3,3).astype("double")
result = predictor.predict({"x": sample})

NameError: name 'X_test2' is not defined

In [36]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"x": tensor2})
result = stub.Predict(request)
result = np.array(result.outputs['y'].double_val).argmax()
print(f'Result activity:  {decode(result)}')
print(f'Actual activity:  {decode(yval2.argmax())}')

Result activity:  walking
Actual activity:  standing
